In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os
import matplotlib.animation as animation
%matplotlib qt5

def smoothTriangle(data, degree):
    triangle=np.concatenate((np.arange(degree + 1), np.arange(degree)[::-1])) # up then down
    smoothed=[]

    for i in range(degree, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(np.sum(point)/np.sum(triangle))
    # Handle boundaries
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed

#defino mi propia funcion de busqueda de minimos 

def minimos_locales(data):
    
    t = np.arange(len(data)) 
    v = data 
 
    dv_dt=(np.diff(v ) )/ (np.diff(t ) )

    a = dv_dt * np.roll(dv_dt , 1)

    tmins = np.argwhere(a < 0)



    return tmins

# Ejercicio 2, item b

In [2]:
path = 'outs/'

filelistvx = sorted(glob(path+'vx*.out'))
filelistvy = sorted(glob(path+'vy*.out'))
filelistvz = sorted(glob(path+'vz*.out'))

filelistwx = sorted(glob(path+'wx*.out'))
filelistwy = sorted(glob(path+'wy*.out'))
filelistwz = sorted(glob(path+'wz*.out'))


nfiles = np.size(filelistvx)
filelistvx

# Spatial resolution
NX = 128
NY = 128
NZ = 128
shape = (NX,NY,NZ)

Hx = []
Hy = []
Hz = []

for i in range(1, nfiles):
    # Reads binary files x
    vx = np.fromfile(filelistvx[i],dtype=np.float32)

    wx = np.fromfile(filelistwx[i],dtype=np.float32)

    # Reads binary files y


    vy = np.fromfile(filelistvy[i],dtype=np.float32)


    wy = np.fromfile(filelistwy[i],dtype=np.float32)

    # Reads binary files z


    vz = np.fromfile(filelistvz[i],dtype=np.float32)


    wz = np.fromfile(filelistwz[i],dtype=np.float32)
        
    #caalculates v.w
    hx = (vx*wx).reshape(shape,order='F')
    hy = (vy*wy).reshape(shape,order='F')
    hz = (vz*wz).reshape(shape,order='F')
    print("helicity has been calculated")
    Hx.append(hx)
    Hy.append(hy)
    Hz.append(hz)
    print("saved helicity at time ", i)
    

helicity has been calculated
saved helicity at time  1
helicity has been calculated
saved helicity at time  2
helicity has been calculated
saved helicity at time  3
helicity has been calculated
saved helicity at time  4
helicity has been calculated
saved helicity at time  5
helicity has been calculated
saved helicity at time  6
helicity has been calculated
saved helicity at time  7
helicity has been calculated
saved helicity at time  8
helicity has been calculated
saved helicity at time  9
helicity has been calculated
saved helicity at time  10
helicity has been calculated
saved helicity at time  11
helicity has been calculated
saved helicity at time  12
helicity has been calculated
saved helicity at time  13
helicity has been calculated
saved helicity at time  14
helicity has been calculated
saved helicity at time  15
helicity has been calculated
saved helicity at time  16
helicity has been calculated
saved helicity at time  17
helicity has been calculated
saved helicity at time  18
h

grafico de un corte en plano x,z de la helicidad a tiempo 0 y a tiempo final:


In [3]:
H3D = np.array(Hx)+np.array(Hy)+np.array(Hz)

plt.close('all')

plt.figure(1)
#plt.imshow(Hx[19][:,:,int(NZ/2)]) #helicity_x in xy at time 19
plt.imshow(H3D[0][:,int(NY/2),:].T) #helicity in xz at time 0
plt.xlabel('x')
plt.ylabel('z')
plt.show()

plt.figure(2)
#plt.imshow(Hx[19][:,:,int(NZ/2)]) #helicity_x in xy at time 19
plt.imshow(H3D[-1][:,int(NY/2),:].T) #helicity in xz at the end
plt.xlabel('x')
plt.ylabel('z')
plt.show()

Animacion de un corte en plano x,z de la helicidad:

In [4]:

fig = plt.figure()
plt.xlabel('x')
plt.ylabel('z')
#agregar barrita de colores

ims = []
for k in range(len(H3D)):
    im = plt.imshow(H3D[k][:,int(NY/2),:].T,animated=True) #helicity in xz at time k
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=10)

plt.colorbar()

# ani.save('helicity_planoxy.gif') 

A medida que el sistema evoluciona, la helicidad se propaga en la dirección z (creo). BUSCAR DAVIDSON.
“Turbulence in rotating, stratified, and electrically conducting fluids“, P.A. Davidson, Cambridge University Press (2013).

Update: lo charle con Nahuel, y me confirmo que la propagacion de la helicidad es en z.

In [7]:
#For later: animación 3D
'''from skimage import data
from scipy import ndimage as ndi
from napari_animation import Animation
import napari'''

'from skimage import data\nfrom scipy import ndimage as ndi\nfrom napari_animation import Animation\nimport napari'

# Ejercicio 2, item c

usando binning del plano x,z (colapsando en la direccion x):

In [71]:
#binning si axis = 0 colapsa las filas, si axis = 1 colapsa las columnas
#quiero colapsar el eje x ---> osea solo quedarme con la dimension z

bins = []
for k in range(len(H3D)):
    bin = np.mean(H3D[k][:,int(NY/2),:] , axis = 0) #helicity colapsada en el eje x  at time k
    bins.append([bin])

fig = plt.figure()
plt.xlabel('mean helicity')
plt.ylabel('z')

ims1 = []
for k in range(len(bins)):
    im, = plt.plot(smoothTriangle(bins[k][0] , 15),animated=True) #helicity in xz at time k smootheado
    ims1.append([im])

ani = animation.ArtistAnimation(fig, ims1, interval=150, blit=True,
                                repeat_delay=10)

#ani.save('mean_helicity_evolution.gif') 

Se ve que hay unos paquetes que se propagan, helicidad neg y pos, con una velocidad definida para las crestas. Puedo calcular maximos y minimos locales a cada timepo para estimar la velocidad de propagacion.


In [25]:
mins = []

for i in range(len(bins)):
    m = minimos_locales( smoothTriangle(bins[i][0] , 15) )
    mins.append(m.reshape((1,len(m)))[0] )

In [73]:
fig = plt.figure()
plt.xlabel('peaks of helicity')
plt.ylabel('z')


ims1 = []
for k in range(len(bins)):
    im, = plt.plot(smoothTriangle(bins[k][0] , 15),animated=True) #helicity in xz at time k smootheado
    ims1.append([im])


ims2 = []
for k in range(len(mins)):
    
    wave_values_k = []
    for j in range(len(mins[k])):
        wave_values_k.append(smoothTriangle( bins[k][0] , 15)[ mins[k][j] ] )
    
    im = plt.scatter( mins[k], wave_values_k , animated=True) #helicity in xz at time k smootheado
    ims2.append([im])

ani1 = animation.ArtistAnimation(fig, ims1, interval=150, blit=True,
                                repeat_delay=10)
    
ani2 = animation.ArtistAnimation(fig, ims2, interval=150, blit=True,
                                repeat_delay=10)


Probaremos con el metodo de calcular $$ Hcm  = \frac{1}{H_{<0 o >0}} \sum_{i=0}^{N} hi.ri $$ 

y calcularle la velocidad a estos objetos: deberian ser aproximadamente las posiciones 'centro de masa' de las helicidades negativas y positivas. Su velocidad en x deberia ser aprox nula, y la vel en z la que buscamos.

In [5]:
print('time , x , y , z : ' ,H3D.shape)

time , x , y , z :  (20, 128, 128, 128)


para cada tiempo hay una matriz de 128x128x128 donde estan las helicidades (cantidad escalar!) para cada (x,y,z). 
\
\
Entonces en cada tiempo, aplico una mascara booleana para obtener los H<0 o H>0:
\
\
(doc. para indexing con boolean masks en numpy: https://docs.scipy.org/doc/numpy-1.15.0/user/basics.indexing.html#boolean-or-mask-index-arrays , \
https://stackoverflow.com/questions/53814869/numpy-multidimensional-boolean-mask)
\
\
la estrategia es multiplicar la mascara booleana por el array original, entonces los falsos dejaran ceros en los elementos correspondientes.

In [6]:
boolneg = H3D < 0

H3Dneg = H3D*boolneg

H3Dneg.shape #chequeo que el shape sea el mismo

(20, 128, 128, 128)

In [7]:
#animacion de helicidad negativa en un corte xz
fig = plt.figure()
plt.xlabel('x')
plt.ylabel('z')
#agregar barrita de colores

ims = []
for k in range(len(H3Dneg)):
    im = plt.imshow(H3Dneg[k][:,int(NY/2),:].T,animated=True) #helicity in xz at time k
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=10)

plt.colorbar()

In [8]:
boolpos = H3D > 0

H3Dpos = H3D*boolpos

H3Dpos.shape #chequeo que el shape sea el mismo

(20, 128, 128, 128)

In [25]:
#animacion de helicidad positiva en un corte xz
fig = plt.figure()
plt.xlabel('x')
plt.ylabel('z')
#agregar barrita de colores

ims = []
for k in range(len(H3Dpos)):
    im = plt.imshow(H3Dpos[k][:,int(NY/2),:].T,animated=True) #helicity in xz at time k
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                repeat_delay=10)

plt.colorbar()

In [10]:
from scipy.ndimage.measurements import center_of_mass

In [11]:
#para cada tiempo calculo centro de masa de Hpos y Hneg

rcmHpos = np.zeros((len(H3D) , 3))
rcmHneg = np.zeros((len(H3D) , 3))

for k in range(len(H3D)):
    rcmHposk = np.array(center_of_mass(H3Dpos[k] ))
    rcmHnegk = np.array(center_of_mass(H3Dneg[k] ))
    
    rcmHpos[k][:] = rcmHposk
    rcmHneg[k][:] = rcmHnegk



In [12]:
rcmHpos

array([[61.73022678, 66.63094509, 56.12558394],
       [61.36675103, 65.86510999, 52.24651953],
       [60.92773609, 65.2052678 , 49.32265801],
       [61.23258303, 65.27771008, 46.79559356],
       [61.73893637, 65.34074109, 44.52975452],
       [62.21405688, 65.49155677, 42.14342052],
       [62.51342491, 65.55346079, 39.6505825 ],
       [62.66298934, 65.59146603, 37.21448496],
       [62.69815214, 65.67731814, 34.78529096],
       [62.73099065, 65.78021928, 32.58599453],
       [62.81624359, 65.88830787, 30.79635244],
       [62.90979795, 66.00012814, 29.42382109],
       [63.05134666, 66.11708442, 28.57729619],
       [63.3228319 , 66.28828444, 28.53621117],
       [63.8769545 , 66.51605786, 30.01221382],
       [64.43582211, 66.54858554, 34.64253968],
       [64.2113007 , 66.16036661, 44.05083714],
       [64.03376603, 65.57037296, 56.48699875],
       [64.67869555, 64.69067376, 67.75272466],
       [65.16320813, 64.08170763, 75.55905673]])

In [14]:
rcmHneg.shape

(20, 3)

In [32]:
#me quedo solamente con la componente en z de rcm de los negativos y positivos
rzneg= rcmHneg[: , -1]
rzpos = rcmHpos[: , -1]

#calculo la velocidad usando dif finita
dt = 0.05 #en segundos
vzneg = np.diff(rzneg)/dt
vzpos = np.diff(rzpos)/dt

voy a visualizar los rcm sobre las animaciones

In [29]:
#animacion de helicidad positiva en un corte xz con su rcmz
fig = plt.figure()
plt.xlabel('x')
plt.ylabel('z')
#agregar barrita de colores

ims1 = []
for k in range(len(H3Dpos)):
    im = plt.imshow(H3Dpos[k][:,int(NY/2),:].T,animated=True) #helicity in xz at time k
    ims1.append([im])


ims2 = []
for k in range(len(rzpos)):
    im = plt.hlines(rzpos[k],0 , 120 ,animated=True) #helicity in xz at time k
    ims2.append([im])

    
ani1 = animation.ArtistAnimation(fig, ims1, interval=100, blit=True,
                                repeat_delay=10)
    
ani2 = animation.ArtistAnimation(fig, ims2, interval=100, blit=True,
                                repeat_delay=10)


plt.colorbar()

In [31]:
#animacion de helicidad negativa en un corte xz con su rcmz
fig = plt.figure()
plt.xlabel('x')
plt.ylabel('z')
#agregar barrita de colores

ims1 = []
for k in range(len(H3Dneg)):
    im = plt.imshow(H3Dneg[k][:,int(NY/2),:].T,animated=True) #helicity in xz at time k
    ims1.append([im])


ims2 = []
for k in range(len(rzneg)):
    im = plt.hlines(rzneg[k],0 , 120 ,animated=True) #helicity in xz at time k
    ims2.append([im])

    
ani1 = animation.ArtistAnimation(fig, ims1, interval=100, blit=True,
                                repeat_delay=10)
    
ani2 = animation.ArtistAnimation(fig, ims2, interval=100, blit=True,
                                repeat_delay=10)


plt.colorbar()

In [39]:
plt.figure()

plt.scatter(np.arange(0 , 1 , 0.05) , [0] + vzneg.tolist()  )
plt.scatter(np.arange(0 , 1 , 0.05) , [0] + vzpos.tolist() )

plt.ylabel('vz del centro de helicidad')
plt.xlabel('t (s)')

Text(0.5, 0, 't (s)')